This code: merges CSV files, shows which days and hours are in the collective database, creates df, converts x, y, values in to date-time and price including VAT and energy tax 2025 (0.1228). it should prompt for energy tax values, allowing to enter future tax values.

it shows the development of price predictions for the period -4 hours till 7 x 24 hours in the future.


In [1]:
# combine all price prediction files into a single database merged_df, which is also saved as CSV file "merged_data_price_preds.csv", located in same folder as this notebook
import os
import pandas as pd
import glob
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the folder containing the CSV files
folder_path = Path("oxygent_data")

# Get all CSV files in the folder
csv_files = list(folder_path.glob("*.csv")) + list(folder_path.glob("*.CSV"))

# Sort files based on timestamp in filename
csv_files.sort(key=lambda x: x.stem.split("_")[-1].replace(":", ""))

# Read and concatenate all CSV files
df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        df_list.append(df)
        logging.info(f"Successfully read {file}")
    except Exception as e:
        logging.error(f"Error reading {file}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)

    # Save the merged data
    output_file = "merged_data_price_preds.csv"
    merged_df.to_csv(output_file, index=False)
    logging.info(f"Merged {len(csv_files)} files into '{output_file}'")
else:
    logging.warning("No files were read successfully.")

2025-05-31 09:50:29,488 - INFO - Successfully read oxygent_data/time_series_data_20250329 00:09 2.csv
2025-05-31 09:50:29,489 - INFO - Successfully read oxygent_data/time_series_data_20250330 02:24 2.csv
2025-05-31 09:50:29,490 - INFO - Successfully read oxygent_data/time_series_data_20250402 08:22 2.csv
2025-05-31 09:50:29,491 - INFO - Successfully read oxygent_data/time_series_data_20250403 02:51 2.csv
2025-05-31 09:50:29,492 - INFO - Successfully read oxygent_data/time_series_data_20250404 05:12 2.csv
2025-05-31 09:50:29,493 - INFO - Successfully read oxygent_data/time_series_data_20250405 03:19 2.csv
2025-05-31 09:50:29,494 - INFO - Successfully read oxygent_data/time_series_data_20250406 03:31 2.csv
2025-05-31 09:50:29,495 - INFO - Successfully read oxygent_data/time_series_data_20250407 03:31 2.csv
2025-05-31 09:50:29,496 - INFO - Successfully read oxygent_data/time_series_data_20250411 12-00.csv
2025-05-31 09:50:29,497 - INFO - Successfully read oxygent_data/time_series_data_202

In [2]:

# Extract dates from the 'timestamp' column
merged_df['date_timestamp'] = pd.to_datetime(merged_df['timestamp']).dt.date
# Extract hour from the 'timestamp' column
merged_df['hour_timestamp'] = pd.to_datetime(merged_df['timestamp']).dt.hour


# Print df headers
print(merged_df)



                 x        y                         timestamp  subarray  \
0       17431164.0  0.08521  2025-03-29T00:09:51.091516+00:00         0   
1       17431200.0  0.07905  2025-03-29T00:09:51.091516+00:00         0   
2       17431236.0  0.07899  2025-03-29T00:09:51.091516+00:00         0   
3       17431272.0  0.07789  2025-03-29T00:09:51.091516+00:00         0   
4       17431308.0  0.07847  2025-03-29T00:09:51.091516+00:00         0   
...            ...      ...                               ...       ...   
745203  17435196.0  0.10000  2025-03-25T19:00:04.963399+00:00         5   
745204  17435232.0  0.11000  2025-03-25T19:00:04.963399+00:00         5   
745205  17435268.0  0.11700  2025-03-25T19:00:04.963399+00:00         5   
745206  17435304.0  0.10400  2025-03-25T19:00:04.963399+00:00         5   
745207  17435340.0  0.09000  2025-03-25T19:00:04.963399+00:00         5   

       date_timestamp  hour_timestamp  
0          2025-03-29               0  
1          2025-03-

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Convert date_timestamp to datetime format
merged_df['date_timestamp'] = pd.to_datetime(merged_df['date_timestamp'])

# Get unique dates
unique_dates = merged_df['date_timestamp'].dt.date.unique()
unique_dates.sort()

# Calculate date range statistics
first_date = min(unique_dates)
last_date = max(unique_dates)
total_days = (last_date - first_date).days + 1
available_days = len(unique_dates)

print(f"Data Range Summary:")
print(f"First date: {first_date}")
print(f"Last date: {last_date}")
print(f"Total days in range: {total_days}")
print(f"Days with data: {available_days}")
print(f"Coverage: {(available_days/total_days)*100:.1f}%")

# Check for missing dates
date_range = pd.date_range(start=first_date, end=last_date, freq='D')
missing_dates = set(date_range.date) - set(unique_dates)

if missing_dates:
    print("\nMissing dates:")
    for date in sorted(missing_dates):
        print(date)

Data Range Summary:
First date: 2025-02-24
Last date: 2025-05-28
Total days in range: 94
Days with data: 78
Coverage: 83.0%

Missing dates:
2025-03-27
2025-04-08
2025-04-09
2025-04-10
2025-05-03
2025-05-04
2025-05-05
2025-05-16
2025-05-17
2025-05-18
2025-05-22
2025-05-23
2025-05-24
2025-05-25
2025-05-26
2025-05-27


In [4]:
# Filter merged_df for dates between April 12, 2025 and May 2, 2025
merged_df = merged_df[
    (merged_df['date_timestamp'] > '2025-04-11') & 
    (merged_df['date_timestamp'] < '2025-05-03')
]


# Get the date range
date_range = pd.to_datetime(merged_df['date_timestamp']).dt.date.unique()
print("Date range in filtered data:")
print(f"Start date: {min(date_range)}")
print(f"End date: {max(date_range)}")
print(f"\nNumber of rows in filtered data: {len(merged_df)}")

Date range in filtered data:
Start date: 2025-04-12
End date: 2025-05-02

Number of rows in filtered data: 21354


In [6]:
# sort df by timestamp, secondly by subarray, thirdly by 'x', to easy interpretation of transformation of x to proper datetime format
merged_df = merged_df.sort_values(by=['timestamp', 'subarray', 'x'])

# ensure conversion from net price to price including taxes..

from datetime import datetime

# Ensure 'x' column is numeric
merged_df['x'] = pd.to_numeric(merged_df['x'], errors='coerce')


# Convert 'x' to a datetime column
merged_df['date_time'] = merged_df['x'].apply(lambda x: datetime.fromtimestamp(x * 100000 / 1000))

# Reorder columns
merged_df = merged_df[['x', 'y', 'timestamp', 'subarray', 'date_timestamp', 'hour_timestamp', 'date_time']]

print(merged_df)




                x       y                         timestamp  subarray  \
9209   17443260.0  0.0851  2025-04-12T00:00:05.000491+00:00         0   
9210   17443296.0  0.0890  2025-04-12T00:00:05.000491+00:00         0   
9211   17443332.0  0.0753  2025-04-12T00:00:05.000491+00:00         0   
9212   17443368.0  0.0750  2025-04-12T00:00:05.000491+00:00         0   
9213   17443404.0  0.0851  2025-04-12T00:00:05.000491+00:00         0   
...           ...     ...                               ...       ...   
30558  17467344.0  0.0610  2025-05-02T00:00:05.000644+00:00         5   
30559  17467380.0  0.0290  2025-05-02T00:00:05.000644+00:00         5   
30560  17467416.0  0.0140  2025-05-02T00:00:05.000644+00:00         5   
30561  17467452.0  0.0010  2025-05-02T00:00:05.000644+00:00         5   
30562  17467488.0  0.0030  2025-05-02T00:00:05.000644+00:00         5   

      date_timestamp  hour_timestamp           date_time  
9209      2025-04-12               0 2025-04-11 01:00:00  
9210 

In [ ]:
'''# Build a mapping of year to energy tax rate
years = merged_df['date_time'].dt.year.unique()
energy_tax_rates = {}
for year in sorted(years):
    if year == 2025:
        energy_tax_rates[year] = 0.1228
    else:
        rate_input = input(f"Enter energy tax rate for year {year}: ")
        try:
            energy_tax_rates[year] = float(rate_input)
        except Exception as e:
            print(f"Invalid input, defaulting energy tax rate for {year} to 0.0")
            energy_tax_rates[year] = 0.0

print("Energy tax rates by year:", energy_tax_rates)'''

In [ ]:

'''# applying 2025 tax rate and VAT to calcuate electricity price including taxes
# Ensure 'y' column is numeric
merged_df['y'] = pd.to_numeric(merged_df['y'], errors='coerce')

merged_df['Price'] = merged_df['y'] * 1.21 + energy_tax_rates[2025]
print(merged_df)'''


In [7]:
from datetime import datetime

# re-Sort merged_df based on the proper datetime column to avoid plotting errors !!
merged_df = merged_df.sort_values(by=['date_timestamp', 'hour_timestamp', 'date_time'])

print("Sorted head values:")
print(merged_df.head(2))
print()
print("Sorted tail values:")

# Save the DataFrame as a CSV file with the current date in the filename
# Define the path to save the CSV file
output_path = Path("../../src/data/price_predictions_oxygent")
output_path.mkdir(parents=True, exist_ok=True)

current_date = datetime.now().strftime("%Y%m%d")
output_filename = output_path / f"Price_Preds_Processed_{current_date}.csv"
merged_df.to_csv(output_filename, index=False)

print(f"DataFrame saved as '{output_filename}'")



Sorted head values:
               x       y                         timestamp  subarray  \
9209  17443260.0  0.0851  2025-04-12T00:00:05.000491+00:00         0   
9233  17443260.0  0.0170  2025-04-12T00:00:05.000491+00:00         1   

     date_timestamp  hour_timestamp           date_time  
9209     2025-04-12               0 2025-04-11 01:00:00  
9233     2025-04-12               0 2025-04-11 01:00:00  

Sorted tail values:
DataFrame saved as '../../src/data/price_predictions_oxygent/Price_Preds_Processed_20250531.csv'
